# Preparing classification datasets

In [1]:
# %config InlineBackend.figure_format='retina'
from ekorpkit import eKonf

eKonf.setLogger("INFO")
print("version:", eKonf.__version__)
print("is notebook?", eKonf.is_notebook())
print("is colab?", eKonf.is_colab())
print("environment variables:")
eKonf.print(eKonf.env().dict())

INFO:ekorpkit.base:IPython version: (6, 9, 0), client: jupyter_client
INFO:ekorpkit.base:Google Colab not detected.


version: 0.1.35+11.gf4a3a1c.dirty
is notebook? True
is colab? False
evironment varialbles:
{'CUDA_DEVICE_ORDER': None,
 'CUDA_VISIBLE_DEVICES': None,
 'EKORPKIT_CONFIG_DIR': '/workspace/projects/ekorpkit-book/config',
 'EKORPKIT_DATA_DIR': None,
 'EKORPKIT_LOG_LEVEL': 'INFO',
 'EKORPKIT_PROJECT': 'ekorpkit-book',
 'EKORPKIT_WORKSPACE_ROOT': '/workspace',
 'KMP_DUPLICATE_LIB_OK': 'TRUE',
 'NUM_WORKERS': 230}


In [2]:
data_dir = "../data/esg"

## Fetch the labeled dataset from the labelstudio server

In [3]:
cfg = eKonf.compose("io/fetcher=labelstudio")
cfg.name = "esg_polarity_labels"
cfg.project_id = 2
ls = eKonf.instantiate(cfg)

INFO:ekorpkit.base:Loaded .env from /workspace/projects/ekorpkit-book/config/.env
INFO:ekorpkit.base:IPython version: (6, 9, 0), client: jupyter_client
INFO:ekorpkit.io.fetch.base:/workspace/.tmp/fetcher/esg_polarity_labels/esg_polarity_labels.parquet already exists. skipping..
INFO:ekorpkit.io.file:Processing [1] files from ['/workspace/.tmp/fetcher/esg_polarity_labels/esg_polarity_labels.parquet']
INFO:ekorpkit.io.file:Loading 1 dataframes from ['/workspace/.tmp/fetcher/esg_polarity_labels/esg_polarity_labels.parquet']
INFO:ekorpkit.io.file:Loading data from /workspace/.tmp/fetcher/esg_polarity_labels/esg_polarity_labels.parquet


## Snorkel LabelModel

In [4]:
model_cfg = eKonf.compose("model/snorkel")
model_cfg.name = "esg_polarity_snorkel"
model_cfg.output_dir = data_dir
model_cfg.verbose = True
snorkel = eKonf.instantiate(model_cfg)

INFO:ekorpkit.base:instantiating ekorpkit.models.snorkel.base.BaseSnorkel...


In [5]:
snorkel.load_data(data=ls.data, test_size=0.2, random_state=12345)

INFO:ekorpkit.models.snorkel.base:Train data: (13936, 7), Test data: (3485, 7)


Train data: (13936, 7)
          id                                               text  annot_id  \
6531   36071  정 장관은 문화적 소외·격차와 관련, '격차를 없애기 위해 촘촘하고 항구적인 시스템...      4674   
13479  20523  특히 기후변화에 대한 전 세계적인 관심이 높아짐에 따라 csr 내에서 온실가스 감축...      6266   
4151   44044  정경유착 및 갑질 근절 등 재벌개혁 범죄수익 환수, 전경련 해체, 불법 재벌총수 처...      1225   
2414   48651  ugv의 하위 인식 시스템은 센서로부터 데이터를 가져와 ugv 주변 환경을 재현하는...      5214   
452    54286  이전의 패키지 요금제에서는 데이터 사용량 증가와 요금제 상향 사이에 괴리가 있었으나...      6865   

       annotator  origin    labels  classes  
6531           5  manual   Neutral        1  
13479          8  manual  Positive        2  
4151           3  manual   Neutral        1  
2414           5  manual   Neutral        1  
452            6  manual  Positive        2  
Test data: (3485, 7)
          id                                               text  annot_id  \
9696   24469  고용노동부가 중심이 되어 저리 대부사업 민관 모태펀드 조성 등을 진행하고 있으나 시...      3132   
13195  20826  '제품 기반 투자 배제'는 담배, 술, 도박, 마약, 대량살상무기 제조업체와 같이 ...

## Writing Labeling Functions

Each crowdworker can be thought of as a single labeling function, as each worker labels a subset of data points, and may have errors or conflicting labels with other workers / labeling functions. Labeling fucntions will simply return the label the worker submitted for a given text, and abstain if they didn't submit a label for it.

### Crowdworker labeling functions

In [6]:
eKonf.viewsource(snorkel.compose_worker_lfs)

    def compose_worker_lfs(self):
        labels_by_annotator = self.data.groupby(self.columns.annotator)
        worker_dicts = {}
        for worker_id in labels_by_annotator.groups:
            worker_df = labels_by_annotator.get_group(worker_id)
            worker_dicts[worker_id] = dict(zip(worker_df.id, worker_df.classes))

        log.info(f"Number of workers: {len(worker_dicts)}")

        def worker_lf(x, worker_dict):
            return worker_dict.get(x.id, self.ABSTAIN)

        def make_worker_lf(worker_id):
            worker_dict = worker_dicts[worker_id]
            name = f"worker_{worker_id}"
            return LabelingFunction(
                name, f=worker_lf, resources={"worker_dict": worker_dict}
            )

        worker_lfs = [make_worker_lf(worker_id) for worker_id in worker_dicts]
        self._worker_lfs = worker_lfs
        return worker_lfs



In [7]:
worker_lfs = snorkel.compose_worker_lfs()

INFO:ekorpkit.models.snorkel.base:Number of workers: 9


In [8]:
snorkel.apply_worker_lfs(worker_lfs)

INFO:ekorpkit.models.snorkel.base:Applying worker lfs to train data
100%|██████████| 13936/13936 [00:00<00:00, 16022.67it/s]
INFO:ekorpkit.models.snorkel.base:Applying worker lfs to test data
100%|██████████| 3485/3485 [00:00<00:00, 15774.94it/s]


In [9]:
summary = snorkel.lf_summary()
summary

INFO:ekorpkit.models.snorkel.base:Training set coverage:  100.0%


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
worker_1,0,"[0, 1, 2]",0.000574,0.000359,0.000000,8,0,1.000000
worker_2,1,"[0, 1, 2]",0.027483,0.012486,0.000431,379,4,0.989556
worker_3,2,"[0, 1, 2]",0.083525,0.030640,0.000287,1162,2,0.998282
worker_4,3,"[0, 1, 2]",0.091131,0.077425,0.001005,1262,8,0.993701
worker_5,4,"[0, 1, 2]",0.375502,0.055755,0.001220,5225,8,0.998471
worker_6,5,"[0, 1, 2]",0.322976,0.229621,0.007176,4457,44,0.990224
worker_7,6,"[0, 1, 2]",0.176521,0.122489,0.003875,2432,28,0.988618
worker_8,7,"[0, 1, 2]",0.379879,0.340413,0.007750,5239,55,0.989611
worker_9,8,"[0, 1, 2]",0.023536,0.023536,0.001005,322,6,0.981707


## Train LabelModel And Generate Probabilistic Labels

In [10]:
snorkel.fit()

100%|██████████| 100/100 [00:00<00:00, 172.39epoch/s]


In [11]:
snorkel.eval()

LabelModel Accuracy for train: 0.995
LabelModel Accuracy for test: 0.994


In [12]:
preds = snorkel.predict()

100%|██████████| 17421/17421 [00:01<00:00, 15467.77it/s]


Data that predictions are different from classes:
          id                                               text  annot_id  \
38     29214  재판부가 선고를 미루며 학부모에게 용서를 빌 기회를 준 것은 이번 사건이 사회적 파...     15298   
55     25177  삼성정밀화학, 폴리실리콘 공장 건설 시동\n삼성정밀화학과 미국 memc사가 합작으로...     15035   
70     24388  그렇게 하기 위해서는 가령 녹색유통채널을 구축하고 녹색소비자에 대한 소비실적에 대해...     13725   
71     24368  이렇게 중소기업의 범위는 비록 복잡하지만 뚜렷하게 정의되어 있으나 중견기업의 범위는...     13711   
78     24266  사회적기업 육성을 위한 자본시장 지원방안\n취약계층에게 사회서비스 또는 일자리를 제...     14983   
...      ...                                                ...       ...   
13281  20730  국내 온실가스 관련 환경법률 개정 동향 안 세 환 연구원\n환경부는 배출권거래제 시...     16226   
13315  20696  따라서 현재 정책의 한계와 문제 점을 적극적으로 확인하고 사회 전체적으로 온실가스 ...       218   
13553  20447  우리나라는 온실가스 감축 비의무국가중 처음으로'저탄소 녹색성장 기본법'제정하였으며,...      1460   
13559  20441  환경부는 2011년 12월 환경정보 공개제도 운영규정 을 고시하고, 녹색기업, 공공...      1457   
13573  20424  문제점 및 개선방안\n먼저 녹색금융, 특히 에너지관련 금융은 효율적인 정책의 수행을...      1443   

       annotator  origin 

In [13]:
snorkel.save_preds(preds, columns = ['id', 'text', "labels"])
eKonf.load_data("esg_polarity_snorkel_preds.parquet", data_dir)

INFO:ekorpkit.io.file:Saving dataframe to ../data/esg/esg_polarity_snorkel_preds.parquet
INFO:ekorpkit.io.file:Processing [1] files from ['esg_polarity_snorkel_preds.parquet']
INFO:ekorpkit.io.file:Loading 1 dataframes from ['../data/esg/esg_polarity_snorkel_preds.parquet']
INFO:ekorpkit.io.file:Loading data from ../data/esg/esg_polarity_snorkel_preds.parquet


,id,text,labels
0,26444,"국내 중소 서버업계 관계자는 '그동안 hpe, 델 등 미국 업체와 경쟁도 힘겨웠는데...",Negative
1,26377,"현대차·모비스·lg이노텍 / 자동차 외장램프용 면조명 확산시트\n현대자동차, 현대모...",Neutral
2,26426,"현대·기아차, 미국에 신모델 투입\n현대·기아자동차가 다음달부터 미국에 신모델을 잇...",Positive
3,26389,"1인승인 ft2는 음성이나 화상 인식 기술을 활용해 운전자의 감정을 읽고, 미리 축...",Positive
4,26388,곧 내놓을 신차는 물론 다소 먼 미래에 출시하기 위해 개발 중인 차가 콘셉트카 형태...,Neutral
...,...,...,...
13625,20371,re100 목적에 도움이 되는 전 세계 또는 국내에서 확실한 영향력을 전달하는 기타...,Positive
13626,20370,"넷째, re100가입 기업은 매년 보고 스프레드시트(cdp 질문지 보고 가능)를 통...",Positive
13627,20369,"가입 요건\nre100 회원은 글로벌 포춘 선정 500대 기업을 포함하여 it, 금...",Positive
13628,20367,"2015 17년 동안 업종별 에너지 사용량을 살펴보면, 사용량 총량은 d35(전기,...",Neutral


## Build a dataset using the data generated by the label model


In [8]:
cfg = eKonf.compose("dataset=dataset_build")
cfg.name = "esg_polarity_kr"
cfg.data_dir = data_dir
cfg.data_file = "esg_polarity_snorkel_data.parquet"
cfg.force.build = True
cfg.pipeline.split_sampling.stratify_on = "labels"
cfg.pipeline.split_sampling.random_state = 123
cfg.pipeline.split_sampling.test_size = 0.2
cfg.pipeline.split_sampling.dev_size = 0.2
cfg.pipeline.reset_index.drop_index = True
cfg.verbose = False
esg_polarity_ds = eKonf.instantiate(cfg)
esg_polarity_ds.persist()

INFO:ekorpkit.pipelines.pipe:Applying pipeline: OrderedDict([('load_dataframe', 'load_dataframe'), ('reset_index', 'reset_index'), ('split_sampling', 'split_sampling')])
INFO:ekorpkit.base:Applying pipe: functools.partial(<function load_dataframe at 0x7f553803bf70>)
INFO:ekorpkit.io.file:Processing [1] files from ['esg_polarity_snorkel_data.parquet']
INFO:ekorpkit.io.file:Loading 1 dataframes from ['../data/esg/esg_polarity_snorkel_data.parquet']
INFO:ekorpkit.io.file:Loading data from ../data/esg/esg_polarity_snorkel_data.parquet
INFO:ekorpkit.base:Applying pipe: functools.partial(<function reset_index at 0x7f553803b1f0>)
INFO:ekorpkit.base:Applying pipe: functools.partial(<function split_sampling at 0x7f5538032dc0>)
INFO:ekorpkit.io.file:Saving dataframe to ../data/esg/esg_polarity_kr/esg_polarity_kr-train.parquet
INFO:ekorpkit.io.file:Saving dataframe to ../data/esg/esg_polarity_kr/esg_polarity_kr-test.parquet
INFO:ekorpkit.io.file:Saving dataframe to ../data/esg/esg_polarity_kr/esg

apply len_bytes to num_bytes:   0%|          | 0/230 [00:00<?, ?it/s]

INFO:ekorpkit.info.stat: >> elapsed time to calculate statistics: 0:00:00.276040
INFO:ekorpkit.base:Using batcher with minibatch size: 10
INFO:ekorpkit.utils.batch.batcher: backend: joblib  minibatch_size: 10  procs: 230  input_split: False  merge_output: True  len(data): 2179 len(args): 5


apply len_bytes to num_bytes:   0%|          | 0/218 [00:00<?, ?it/s]

INFO:ekorpkit.info.stat: >> elapsed time to calculate statistics: 0:00:00.250046
INFO:ekorpkit.base:Using batcher with minibatch size: 12
INFO:ekorpkit.utils.batch.batcher: backend: joblib  minibatch_size: 12  procs: 230  input_split: False  merge_output: True  len(data): 2724 len(args): 5


apply len_bytes to num_bytes:   0%|          | 0/227 [00:00<?, ?it/s]

INFO:ekorpkit.info.stat: >> elapsed time to calculate statistics: 0:00:00.253649
INFO:ekorpkit.io.file:Saving dataframe to ../data/esg/esg_polarity_kr/esg_polarity_kr-train.parquet
INFO:ekorpkit.io.file:Saving dataframe to ../data/esg/esg_polarity_kr/esg_polarity_kr-dev.parquet
INFO:ekorpkit.io.file:Saving dataframe to ../data/esg/esg_polarity_kr/esg_polarity_kr-test.parquet
